## First Notes

Please do not run the file as whole, as some cells include some irrelevant code that was created for testing some examples. The cells to be ignored are noted by a **BEGIN:** and **END;** notations.

### Modules Import

In [6]:
from __future__ import print_function
from rdkit import Chem

### Reading data

Reading a set of molecules an `SDMolSupplier` from rdkit.<br>
In the [getting started](https://www.rdkit.org/docs/GettingStartedInPython.html) documentation there're two these two different suppliers, but I haven't looked up the difference yet:
* `rdkit.Chem.rdmolfiles.SDMolSupplier`
* `rdkit.Chem.rdmolfiles.SmilesMolSupplier`

In [7]:
## Read the file
supplier = Chem.SDMolSupplier('data/cas_4337.sdf')

In [8]:
type(supplier)

rdkit.Chem.rdmolfiles.SDMolSupplier

In [38]:
mol = supplier[97]
mol
# list(mol.GetPropNames())
mol.GetProp("Ames test categorisation")

'mutagen'

In [9]:
## for each molecule, get the number of atoms in it
for mol in supplier:
    print(mol.GetNumAtoms())

25
13
20
10
10
14
18
24
10
24
18
26
18
13
9
17
13
23
6
10
11
13
14
12
21
9
13
13
16
28
21
30
10
10
17
19
14
17
15
11
17
39
18
16
22
17
10
14
5
21
22
4
29
14
20
8
15
18
13
23
9
12
15
9
12
13
8
22
26
13
8
14
19
6
14
14
20
9
12
17
9
21
18
11
18
9
24
51
16
19
13
18
3
21
19
21
12
40
27
18
16
15
8
27
7
26
13
20
12
7
10
9
21
21
13
11
9
22
4
8
5
21
9
20
15
13
37
7
25
14
15
22
16
6
22
10
17
21
21
7
8
15
6
14
25
21
16
10
17
13
22
16
19
29
15
25
22
5
37
33
10
17
102
21
21
18
18
20
11
21
14
11
9
9
23
21
25
8
14
5
14
25
5
12
18
39
18
10
24
13
11
19
13
22
23
19
19
17
20
5
20
19
14
19
8
9
11
18
16
21
8
36
23
14
16
4
12
6
15
20
11
10
20
25
15
6
17
20
18
13
18
16
8
22
13
10
7
25
23
9
16
14
13
9
5
5
18
12
17
20
18
10
12
21
34
3
9
18
7
35
25
15
10
9
16
22
38
7
11
9
19
24
5
12
8
8
20
22
6
11
13
23
12
23
18
20
11
14
20
6
20
40
11
43
16
52
18
13
23
11
13
12
11
18
7
8
10
19
11
8
11
26
9
11
11
16
21
18
15
13
22
18
13
13
19
9
14
14
19
15
5
10
14
10
10
10
16
12
15
16
13
19
9
19
17
15
10
20
16
21
8
12
14
14
22
1

9
5
7
18
16
9
7
12
17
10
19
16
12
24
13
22
15
12
19
7
15
9
14
22
21
11
11
17
25
18
10
15
5
8
17
11
11
6
15
18
5
11
9
36
7
20
23
14
12
11
24
23
11
13
17
18
17
19
19
14
18
12
16
9
15
11
7
17
20
6
12
16
23
17
16
9
14
12
12
8
11
21
15
15
19
15
20
11
12
20
33
28
6
19
19
9
13
6
13
9
108
13
2
4
24
17
17
8
17
24
16
19
17
13
12
19
10
20
22
12
14
7
18
14
24
32
27
7
11
17
10
15
20
49
26
4
30
17
19
12
9
17
12
19
9
10
16
41
20
19
19
15
18
21
5
24
8
13
21
21
11
9
19
11
18
11
30
20
12
6
26
37
15
16
22
4
18
5
26
17
28
17
14
22
8
30
18
10
17
24
15
26
20
15
16
18
11
12
13
15
18
17
14
8
61
28
17
14
11
19
16
6
21
11
10
9
7
20
13
22
19
6
17
19
13
15
19
23
7
12
15
20
12
24
9
29
15
22
6
11
12
10
25
18
18
12
24
19
7
17
30
16
16
17
21
21
14
23
13
9
14
18
24
18
11
14
7
28
14
12
23
8
14
13
6
16
8
11
24
14
17
13
20
20
20
12
19
27
22
19
20
23
19
24
14
5
21
4
7
12
6
13
16
11
19
20
21
17
10
17
9
16
25
19
10
7
12
12
17
14
11
22
19
24
13
12
46
26
19
12
31
15
18
18
7
17
28
9
17
17
32
11
16
14
16
32
15
13
11
26
14
19
21

We have 4337 molecules in the data set:

In [10]:
len(supplier)

4337

A good practice is to test each molecule to see if it was correctly read before working with it:

In [11]:
for mol in supplier:
    if mol is None:
        print("a None molecule was found!")
        
## if the output is empty the data is fine.

<br>
<br>
<br>
<br>
<br>
<br>

---
*The following code can be ignored, cause this was generated for my own interest and testing.*

---
BEGIN:

In [54]:
m = Chem.MolFromSmiles('C1OC1')
for atom in m.GetAtoms():
    print(atom.GetAtomicNum())

print(m.GetBonds()[1].GetBondType())

6
8
6
SINGLE


END;

----
<br>
<br>
<br>
<br>
<br>
<br>

### Calculating Morgan fingerprints

We're going to go trough two steps:<br>
first, we'll extract the training samples, which are going to be the values in the bits dictionary (`info`).<br>
Second, extracting the labels for each molecule where a Morgan fingerprint can be located.

#### Training Samples

In [12]:
import numpy as np

In [13]:
from rdkit.Chem import AllChem

> **\_\_TASK\_\_:** for each molecule calculate MorganFingerprints (with radius <b>3</b>) and size **~2048** (rdkit has also a nice easy function for that)

In [69]:
info = {} # this variable will be mutated in the next function (very similar to pointers in C)
# fingerprints = [list(AllChem.GetMorganFingerprint(mol, 3, bitInfo=info)) for mol in supplier]

fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048, bitInfo=info) for mol in supplier]
# fingerprints=[list(fp) for fp in fingerprints_asBitVector]
fingerprints = np.array(fingerprints)

In [70]:
fingerprints.shape

(4337, 2048)

##### Atoms contributing to the activation of Morgan fingerprints

> **\_\_TASK\_\_:** Important! when you calculate the Fingerprints, save which atoms where responsible for the activation of the fingerprint (rdkit can also do that)

###### `bitsInfo` Explaining bits from Morgan Fingerprints

Information is available about the atoms that contribute to particular bits in the Morgan fingerprint via the bitInfo argument. The dictionary provided is populated with one entry per bit set in the fingerprint, <u>the keys are the bit ids</u>, <u>the values are lists of (atom index, radius) tuples</u>.

And there we go. Now have succefully extracted the training samples that are ready to be passed to the model.

So let's have a look on the dictionary in `info`:

In [72]:
info

{97: ((14, 0),),
 191: ((10, 1),),
 263: ((4, 2),),
 314: ((7, 1), (9, 1)),
 325: ((15, 3),),
 336: ((2, 2), (3, 2)),
 389: ((17, 2),),
 484: ((6, 3), (8, 3)),
 606: ((11, 2),),
 650: ((5, 0), (7, 0), (9, 0)),
 689: ((1, 1),),
 703: ((12, 2), (13, 2)),
 807: ((1, 0),),
 811: ((6, 2), (8, 2)),
 856: ((11, 3),),
 905: ((0, 3),),
 993: ((0, 2),),
 1019: ((0, 0),),
 1034: ((2, 1), (3, 1)),
 1060: ((6, 1), (8, 1)),
 1077: ((10, 2),),
 1088: ((15, 1), (16, 1), (17, 1)),
 1114: ((6, 0), (8, 0)),
 1152: ((4, 0),),
 1199: ((15, 2), (16, 2)),
 1216: ((1, 3),),
 1327: ((10, 3),),
 1380: ((2, 0), (3, 0), (10, 0), (11, 0)),
 1460: ((12, 3),),
 1642: ((4, 3),),
 1645: ((11, 1),),
 1682: ((1, 2),),
 1717: ((14, 1),),
 1750: ((12, 1), (13, 1)),
 1771: ((0, 1),),
 1816: ((4, 1),),
 1873: ((12, 0), (13, 0), (15, 0), (16, 0), (17, 0)),
 1917: ((5, 1),),
 1947: ((17, 3),),
 2037: ((2, 3),)}

<br>

Interpreting the results above:

```
97: ((14, 0),),
191: ((10, 1),),
314: ((7, 1), (9, 1)),
```

* bit 97 is set once: by atom 14 with radius 0.
* bit 191 is set also once: by atom 10, with radius 1.
* bit 314, on the other hand, is set twice: once by atom 7 and once with atom 9, each with radius 1.


<br>
<br>
<br>
<br>

---
*The following code can be ignored, cause this was generated for my own interest and testing.*

---
BEGIN:

In the following I'd like to draw one molecule in order to locate the Morgan fingerprint, together with the atoms and radius. So let's draw the molecule first, then let's find out what the following values correspond to:<br>
The first entry in the `info` dictionary is `97: ((14, 0),),`, so we'll see what that means.<br>
Lastly, I'll repeat the same process but with molecule number 314 whose fingerprints are `314: ((7, 1), (9, 1)),`


In [100]:
from rdkit.Chem import Draw

END;

----

<br>
<br>
<br>
<br>
<br>
<br>


#### Training Labels

Now that we've got the the bits ready as training samples (i.e. features) for the model, we still need to extract the label to have a target for our predictions.

The property that we want to extract out of the list of prooperties, is whether or not the 'Ames test Categorisation' a mutagen.
* 1 for mutagen 
* 0 for nonmutagen

In [73]:
training_labels = []
for mol in supplier:
    if mol.GetProp("Ames test categorisation") == "mutagen":
        training_labels.append(1)
    else:
        training_labels.append(0)

training_labels = np.array(training_labels)

In [75]:
len(train_labels)

4337

Now, we've got the whole training set ready. Let's move on to extracting the validation set.

#### Validation Set

In order to select a 'valid' validation, we have randomly picked the following indices of molecules.<br>
Additionally, we'd still need to see how many fingerprints are expressed in this set of molecules.

In the following, I'll be using the alias `val_`, for anything that's related to the validation set, in order to avoid length variables names.

In [46]:
## KAREEM: these molecules I got from Kristina!
val_mol_ids = [6,   10,   29,   32,   42,   58,   72,   83,   98,  100,  128, 
        145,  148,  168,  171,  205,  208,  237,  244,  285,  290,  291,
         300,  312,  332,  334,  335,  347,  356,  369,  371,  377,  407,
         424,  456,  458,  470,  472,  486,  514,  515,  528,  557,  563,
         599,  610,  616,  628,  640,  701,  704,  722,  764,  794,  818,
         821,  840,  850,  856,  859,  874,  878,  882,  898,  901,  925,
         936,  945,  957,  974,  977, 1013, 1019, 1030, 1038, 1047, 1049,
        1072, 1073, 1100, 1159, 1168, 1187, 1190, 1194, 1201, 1202, 1233,
        1247, 1258, 1264, 1273, 1283, 1288, 1300, 1302, 1319, 1339, 1349,
        1402, 1413, 1416, 1422, 1426, 1435, 1454, 1465, 1483, 1502, 1513,
        1515, 1520, 1548, 1576, 1604, 1606, 1621, 1650, 1695, 1696, 1711,
        1714, 1716, 1725, 1743, 1746, 1752, 1780, 1788, 1794, 1799, 1813,
        1826, 1866, 1886, 1901, 1903, 1921, 1929, 1940, 1969, 1970, 1997,
        1998, 2008, 2010, 2011, 2018, 2023, 2046, 2060, 2064, 2080, 2081,
        2131, 2171, 2182, 2203, 2212, 2224, 2231, 2241, 2246, 2283, 2294,
        2295, 2297, 2327, 2329, 2331, 2349, 2357, 2360, 2365, 2397, 2413,
        2417, 2418, 2421, 2448, 2467, 2510, 2516, 2528, 2533, 2549, 2562,
        2601, 2604, 2606, 2609, 2611, 2632, 2644, 2653, 2677, 2682, 2685,
        2692, 2703, 2708, 2714, 2719, 2726, 2732, 2759, 2761, 2776, 2780,
        2817, 2818, 2829, 2837, 2857, 2858, 2884, 2899, 2902, 2905, 2911,
        2939, 2975, 2977, 2986, 3007, 3009, 3018, 3024, 3038, 3066, 3087,
        3098, 3107, 3117, 3122, 3139, 3157, 3161, 3164, 3217, 3223, 3233,
        3263, 3265, 3271, 3290, 3295, 3307, 3313, 3317, 3321, 3382, 3384,
        3388, 3400, 3409, 3412, 3419, 3423, 3449, 3470, 3487, 3488, 3503,
        3509, 3511, 3539, 3562, 3626, 3637, 3654, 3662, 3663, 3668, 3671,
        3688, 3689, 3695, 3710, 3726, 3743, 3744, 3782, 3791, 3794, 3808,
        3809, 3841, 3849, 3874, 3910, 3912, 3925, 3945, 3950, 3958, 3959,
        3962, 3964, 3967, 3978, 3993, 4009, 4010, 4055, 4057, 4085, 4089,
        4096, 4099, 4107, 4112, 4129, 4135, 4151, 4155, 4196, 4209, 4216,
        4234, 4236, 4251, 4267, 4283, 4317, 4326, 4335

]

In [59]:
val_info = {}
for id in val_mol_ids:
    mol = supplier[id]
    AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048, bitInfo=val_info)

# for key in val_info.keys():
#     if val_molecules[key].GetProp("Ames test categorisation") == "mutagen":
#         validation_labels.append(1)
#     else:
#         validation_labels.append(0)
        
# validation_labels = np.array(validation_labels)

In [64]:
AllChem.GetMorganFingerprintAsBitVect(supplier[6], 3, nBits=2048, bitInfo=val_info)

In [65]:
val_info

{27: ((1, 3),),
 64: ((0, 2),),
 152: ((7, 2),),
 164: ((4, 1),),
 184: ((9, 2),),
 189: ((12, 3),),
 191: ((9, 1),),
 485: ((16, 2),),
 486: ((2, 1),),
 552: ((3, 1),),
 636: ((12, 2),),
 650: ((6, 0), (10, 0)),
 675: ((7, 0),),
 703: ((13, 2),),
 715: ((11, 0),),
 725: ((1, 2),),
 728: ((17, 0),),
 753: ((10, 1),),
 766: ((14, 2),),
 767: ((15, 3),),
 805: ((0, 3),),
 807: ((2, 0),),
 838: ((11, 1),),
 864: ((16, 3),),
 875: ((1, 1), (12, 1)),
 909: ((14, 1),),
 957: ((8, 2),),
 1039: ((15, 2),),
 1088: ((15, 1),),
 1146: ((13, 3),),
 1152: ((5, 0),),
 1195: ((8, 1),),
 1319: ((3, 3),),
 1326: ((2, 3),),
 1380: ((0, 0), (4, 0), (9, 0), (14, 0)),
 1385: ((0, 1),),
 1527: ((3, 2),),
 1567: ((7, 1),),
 1608: ((4, 3),),
 1631: ((17, 1),),
 1669: ((4, 2),),
 1679: ((5, 3),),
 1749: ((9, 3),),
 1750: ((13, 1), (16, 1)),
 1795: ((2, 2),),
 1816: ((5, 1),),
 1873: ((1, 0), (3, 0), (12, 0), (13, 0), (15, 0), (16, 0)),
 1917: ((6, 1),),
 1963: ((8, 0),),
 1978: ((5, 2),)}

In [47]:
val_info = {}
val_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048, bitInfo=val_info) for mol in val_molecules]
validation_samples = np.array(list(val_info.values()))

In [58]:
val_info.keys()

dict_keys([1056, 1, 1475, 837, 294, 1057, 1544, 650, 1383, 494, 80, 1650, 979, 725, 1601, 695, 1208, 283])

## Creating The model